# Many experiments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from experiment import main

n_instances = 1
n_samples = 1
n_hetero_vars = 2
n_periods = 2
n_units = 1000
n_x = 10
res = main(n_periods=n_periods, n_units=n_units, n_treatments=1, n_x=n_x, s_x=2, s_t=2,
           n_hetero_vars=n_hetero_vars, 
           n_instances=n_instances, inst_seed=0,
           n_samples=n_samples, sample_seed=0,
           max_poly_degree=2, high_dim=True,
           verbose=1)

In [ ]:
res[0][0]['true']['pival']

In [ ]:
res[0][0]['reg']['pival']

In [ ]:
res[0][0]['ols']['pival']

In [ ]:
res[0][0]['oracle_ols']['pival']

In [ ]:
import numpy as np
[(key, value) for key, value in res[0][0]['true']['params'][0].items() if np.abs(value) > 0]

In [ ]:
res[0][0]['ols']['params'][0]

In [ ]:
import numpy as np
[(key, value) for key, value in res[0][0]['true']['params'][1].items() if np.abs(value) > 0]

In [ ]:
res[0][0]['ols']['params'][1]

# Experiments with non-parametric heterogeneity

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
from experiment_hetero import experiment

fn_list = ['abs', 'linear', 'square', 'step', 'steplinear', '3dpoly']
results = {}
for nonlin_fn in fn_list:
    results[nonlin_fn] = experiment(n_periods=2, n_units=10000,
                       n_x=10, s_x=2, s_t=2,
                       hetero_strenth=2.0, n_hetero_vars=1,
                       autoreg=1.0, nonlin_fn=nonlin_fn,
                       instance_seed=0, sample_seed=0,
                       verbose=1, max_poly_degree=2, store_preds=True)

In [ ]:
import joblib
joblib.dump(results, 'all_plots.jbl')

In [ ]:
import joblib
results = joblib.load('all_plots.jbl')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

period = 0
key = 'softmaxcv'
plt.figure(figsize=(15,10))
for it, nonlin_fn in enumerate(fn_list):
    method_it = 0
    d = results[nonlin_fn][key]
    rmse = d[period]['rmse']
    plt.subplot(2, 3, it + 1)
    plt.title(f'{key}: fn={nonlin_fn}, period={period}, rmse={rmse:.3f}')
    plt.scatter(d[period]['xhet'][:, 0], d[period]['true'])
    plt.scatter(d[period]['xhet'][:, 0], d[period]['pred'])
plt.savefig(f'period{period}.png', dpi=600)
plt.show()

In [ ]:
import pandas as pd
import numpy as np

nonlin_fn = 'abs'
alldfs = {}
for period in range(2):
    table = {}
    for key, d in results[nonlin_fn].items():
        table[key] = {'rmse': d[period]['rmse'], 'mae': d[period]['mae']}
    alldfs[f'period={period}'] = pd.DataFrame(table)

allperiods = pd.concat(alldfs)

for row, col in enumerate(np.argmin(allperiods.values, axis=1)):
    val = allperiods.iloc[row, col]
    allperiods.iloc[row, col] = r'\textbf{' + "{:.3f}".format(val) + r'}'

allperiods.transpose()